Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices

import lab9_lib

In [2]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

11110110101111000101100001011010001101101110110101: 9.11%
01010100001010010000100111010001110010010001011000: 15.56%
10001000110101011101111010100010111100010010000101: 7.33%
11101011100111110001101001001100101010111111011100: 9.33%
10011000110010100011001101010111101010001110010110: 7.33%
01010000100100111110000011000111111000101010001010: 7.34%
10000111010101101000010100101111011101001111000011: 15.33%
10001000011001100110001101111101101111101101011011: 7.33%
00011101001101000111110000011111100010011100100111: 31.36%
10100010101011101011011001000101000111000010011101: 15.34%
10
